In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import recordlinkage as rd
import fuzzymatcher
from difflib import SequenceMatcher
from fuzzywuzzy import process

In [667]:
#read the data
df = pd.read_excel('/home/dr/Downloads/Health Facility Assessment.xlsx')
data = df.copy() #create a copy of the data
data.head()

,Unnamed: 0,number,form.health_centre_information.facility_name,form.facility_gps,form.health_centre_information.location_information.region_province,form.health_centre_information.location_information.district,form.health_centre_information.location_information.chiefdom,form.health_centre_information.location_information.facility_location,form.health_centre_information.facility_type,form.health_centre_information.facility_type_other,...,form.grp_infection_prevention_and_control.ql_water_sanitation_and_hygiene.score_infection_prevention_and_control_water_sanitation_and_hygiene,form.grp_infection_prevention_and_control.ql_water_sanitation_and_hygiene.score_max_infection_prevention_and_control_water_sanitation_and_hygiene,form.grp_infection_prevention_and_control.ql_disinfection_and_sterilization.score_infection_prevention_and_control_disinfection_and_sterilization,form.grp_infection_prevention_and_control.ql_disinfection_and_sterilization.score_max_infection_prevention_and_control_disinfection_and_sterilization,form.grp_infection_prevention_and_control.score_infection_prevention_and_control,form.grp_infection_prevention_and_control.score_max_infection_prevention_and_control,form.question1.score_logistics_patient_and_sample_transfer,form.question1.score_max_logistics_patient_and_sample_transfer,form.score_total,form.score_max_total
0,0,0,William hospital,"60.234598, -90.157892",Southern,Pujehun,New York,Jasonfort,community_health_post,---,...,4,5,1,5,10.5,27,19.5,44,48.5,102.0
1,1,1,Kathryn Hospital/CHC,"21.2454855, -107.509120",Southern,Bo,Arizona,West Jamesshire,community_health_post,---,...,1.5,5,0,5,5.5,27,21,44,35.5,102.0
2,2,2,Jessica Hospital,"-0.178657, -9.190029",Southern,Bo,New Mexico,Robertsside,community_health_post,---,...,2,5,0,5,10,27,33.5,44,55.5,102.0
3,3,3,Lauren CHC,"35.856989, -87.512790",Southern,Bonthe,Missouri,Port Christopher,mch_post,---,...,4,5,2,5,12,27,34.5,44,57.5,102.0
4,4,4,Tammy CHC,"54.7148835, 150.744813",Southern,Bonthe,Virginia,Reneeton,community_health_post,---,...,4,5,3,5,12,27,29.5,44,55.0,102.0


In [608]:
#read master data
m_df = pd.read_excel('/home/dr/Downloads/Master Health Facility.xlsx')
m_data = m_df.copy()
m_data.head()

,Unnamed: 0,Number,Region_Province,District,Chiefdom,Facility_name,Type,Source,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,1,Southern,Bo district,Badjia,Ngelehun CHC,CHC,MoHS,Unnamed: 8,Unnamed: 9,Unnamed: 10
1,0.0,2,Southern,Bo district,Florida,Victoria CHP,MCHP,MoHS,NaN,NaN,NaN
2,1.0,3,Southern,Bo district,Louisiana,Kristina CHC,CHC,MoHS,NaN,NaN,NaN
3,2.0,4,Southern,Bo district,Louisiana,James Regional Hospital,MCHP,MoHS,NaN,NaN,NaN
4,3.0,5,Southern,Bo district,Louisiana,Jake Regional Hospital,MCHP,MoHS,NaN,NaN,NaN


In [609]:
#drop redudant columns and empty columns
m_data = m_data.drop(['Unnamed: 0','Unnamed: 10','Unnamed: 8','Unnamed: 9'],axis=1)
m_data = m_data.dropna(axis=1, how='all')

In [8]:
m_data.head()

,Number,Region_Province,District,Chiefdom,Facility_name,Type,Source
0,1,Southern,Bo district,Badjia,Ngelehun CHC,CHC,MoHS
1,2,Southern,Bo district,Florida,Victoria CHP,MCHP,MoHS
2,3,Southern,Bo district,Louisiana,Kristina CHC,CHC,MoHS
3,4,Southern,Bo district,Louisiana,James Regional Hospital,MCHP,MoHS
4,5,Southern,Bo district,Louisiana,Jake Regional Hospital,MCHP,MoHS


In [668]:
#drop the Unnamed:0 column, redudant excel column
data=data.drop('Unnamed: 0',axis=1)

In [669]:
#get the number of rows and columns
data.shape

(145, 204)

In [612]:
#check the data types of the columns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Columns: 204 entries, number to form.score_max_total
dtypes: float64(2), int64(1), object(201)
memory usage: 231.2+ KB


Handling Missing values
        drop columns that are empty
        drop columns with more than 90% missing values

In [12]:
#highlight columns with missing values
dd=[]
for i in data.isnull().sum():
    if i>0:
        dd.append(i)

In [13]:
#number of columns with missing values
len(dd)

122

In [670]:
#replace '---' with null values
data=data.replace('---', np.nan)

In [671]:
#drop columns that are 100% empty
data = data.dropna(axis =1,how='all')
data.shape

(145, 185)

In [16]:
data.isnull().sum().sort_values(ascending=False)

form.health_centre_information.managing_authority_other                                          143
form.health_centre_information.group_number_employed.list_other.other_staff_cadre                 72
form.ql_logistics_patient_and_sample_transfer.log_tracer_drugs                                    45
form.question10.log_chlorhexidine_7                                                               38
form.question10.log_folic                                                                         26
                                                                                                ... 
form.health_centre_information.group_number_employed.list_other.other_number_in_post               2
form.health_centre_information.group_number_employed.list_other.other_number_present_at_visit      2
form.facility_gps                                                                                  0
form.sections_to_review                                                                    

In [672]:
#drop columns with above 90% missing values
thresh = int(((100-90)/100)* data.shape[0]+1)
data = data.dropna(axis=1,thresh=thresh)
data.shape

(145, 184)

In [673]:
#rename columns
cols =[]
for i in data.columns.tolist()[0:34]:
    cols.append(i.split('.')[-1])

for i in data.columns.tolist()[34:51]:
    cols.append('_'.join(i.split('.')[-2:]))

for i in data.columns.tolist()[51:185]:
    cols.append(i.split('.')[-1])

data.columns=cols    

In [674]:
data[data.columns[11:32]]=data[data.columns[11:32]].astype('float')

In [675]:
data[data.columns[152:184]]=data[data.columns[152:184]].astype('float')

In [676]:
data[data.columns[34:50]] = data[data.columns[34:50]].astype('float')

In [677]:
#drop rows with empty facility name
data = data.dropna(subset=['facility_name'])
data.shape

(143, 184)

In [621]:
data.head(10)

,number,facility_name,facility_gps,region_province,district,chiefdom,facility_location,facility_type,managing_authority,setting,...,score_infection_prevention_and_control_water_sanitation_and_hygiene,score_max_infection_prevention_and_control_water_sanitation_and_hygiene,score_infection_prevention_and_control_disinfection_and_sterilization,score_max_infection_prevention_and_control_disinfection_and_sterilization,score_infection_prevention_and_control,score_max_infection_prevention_and_control,score_logistics_patient_and_sample_transfer,score_max_logistics_patient_and_sample_transfer,score_total,score_max_total
0,0,William hospital,"60.234598, -90.157892",Southern,Pujehun,New York,Jasonfort,community_health_post,government_public,peri_urban,...,4.0,5.0,1.0,5.0,10.5,27.0,19.5,44.0,48.5,102.0
1,1,Kathryn Hospital/CHC,"21.2454855, -107.509120",Southern,Bo,Arizona,West Jamesshire,community_health_post,government_public,rural,...,1.5,5.0,0.0,5.0,5.5,27.0,21.0,44.0,35.5,102.0
2,2,Jessica Hospital,"-0.178657, -9.190029",Southern,Bo,New Mexico,Robertsside,community_health_post,government_public,rural,...,2.0,5.0,0.0,5.0,10.0,27.0,33.5,44.0,55.5,102.0
3,3,Lauren CHC,"35.856989, -87.512790",Southern,Bonthe,Missouri,Port Christopher,mch_post,government_public,peri_urban,...,4.0,5.0,2.0,5.0,12.0,27.0,34.5,44.0,57.5,102.0
4,4,Tammy CHC,"54.7148835, 150.744813",Southern,Bonthe,Virginia,Reneeton,community_health_post,government_public,peri_urban,...,4.0,5.0,3.0,5.0,12.0,27.0,29.5,44.0,55.0,102.0
6,6,Bryan Clinic,"61.8040035, 28.191714",Southern,Pujehun,Michigan,Adammouth,community_health_post,government_public,peri_urban,...,1.0,5.0,2.0,5.0,8.5,27.0,21.0,44.0,53.0,102.0
7,7,Cheryl Hospital/CHC,"86.9708915, 173.527045",Southern,Bonthe,Nevada,Bryanchester,community_health_centre,government_public,peri_urban,...,3.0,5.0,1.0,5.0,7.0,27.0,15.0,44.0,38.0,102.0
8,8,Brandi hospital,"77.8973365, 70.112618",Southern,Moyamba,Florida,Port Kristinville,community_health_centre,government_public,peri_urban,...,2.5,5.0,4.0,5.0,19.5,27.0,25.0,44.0,65.5,102.0
9,9,Michael Regional Hospital,"-24.700264, 79.460077",Northern,Koinadugu,Arkansas,South Adam,mch_post,government_public,peri_urban,...,0.0,5.0,0.0,5.0,0.0,27.0,17.0,44.0,17.0,102.0
10,10,Michael Hospital,"21.108351, -60.068170",Northern,Koinadugu,Georgia,Harrisburgh,community_health_centre,government_public,peri_urban,...,4.0,5.0,3.5,5.0,20.5,27.0,30.5,44.0,76.5,102.0


Data Uniqueness

In [678]:
#highlight the unique values in each column and insert them into a dictionary
uniq_cols = data.nunique().to_dict()

In [679]:
#get the data types of the columns
data_types = data.dtypes.to_dict()

In [680]:
#find numeric columns with little variability
dd = []
for i,j in uniq_cols.items():
   if j<2:
    for k,v in data_types.items():
        if v =='float64':
            if i ==k:
                dd.append(i)


In [681]:
#get the summary statistics of the highlighted columns to affirm no variability
data[dd].describe()

,score_max_human_resources,score_max_information_education_communication,score_max_surveillance,score_max_triage_and_early_recognition,score_max_chw,score_max_isolation,score_max_infection_prevention_and_control_ppe,score_max_infection_prevention_and_control_ppe_plan,score_max_infection_prevention_and_control_waste_collection_and_disposal,score_max_infection_prevention_and_control_water_sanitation_and_hygiene,score_max_infection_prevention_and_control_disinfection_and_sterilization,score_max_infection_prevention_and_control,score_max_logistics_patient_and_sample_transfer
count,132.0,127.0,128.0,132.0,130.0,130.0,125.0,125.0,125.0,125.0,125.0,125.0,125.0
mean,5.0,7.0,4.0,5.0,6.0,4.0,9.0,4.0,4.0,5.0,5.0,27.0,44.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,5.0,7.0,4.0,5.0,6.0,4.0,9.0,4.0,4.0,5.0,5.0,27.0,44.0
25%,5.0,7.0,4.0,5.0,6.0,4.0,9.0,4.0,4.0,5.0,5.0,27.0,44.0
50%,5.0,7.0,4.0,5.0,6.0,4.0,9.0,4.0,4.0,5.0,5.0,27.0,44.0
75%,5.0,7.0,4.0,5.0,6.0,4.0,9.0,4.0,4.0,5.0,5.0,27.0,44.0
max,5.0,7.0,4.0,5.0,6.0,4.0,9.0,4.0,4.0,5.0,5.0,27.0,44.0


In [682]:
#drop numeric columns with no variability
data = data.drop(dd,axis=1)
data.shape

(143, 171)

Data Validity

In [683]:
#split gps data to longitude and latitude
data['latitude']=data['facility_gps'].str.split(',',expand=True)[0]
data['longitude'] = data['facility_gps'].str.split(',',expand=True)[1]
data = data.drop('facility_gps',axis=1)

In [684]:
data['other_staff_cadre'].value_counts()

TBA                                                                               24
Nursing Aide                                                                       5
TBAs                                                                               5
Community Health Workers                                                           4
TBA's                                                                              3
nursing aide                                                                       3
Porter                                                                             2
CHWs                                                                               2
security                                                                           2
CHW                                                                                2
1T.B.A                                                                             1
TBA and Security                                                 

In [685]:
data=data.drop('other_staff_cadre',axis=1)

In [686]:
data.head()

,number,facility_name,region_province,district,chiefdom,facility_location,facility_type,managing_authority,setting,outpatient_only,...,score_infection_prevention_and_control_ppe_plan,score_infection_prevention_and_control_waste_collection_and_disposal,score_infection_prevention_and_control_water_sanitation_and_hygiene,score_infection_prevention_and_control_disinfection_and_sterilization,score_infection_prevention_and_control,score_logistics_patient_and_sample_transfer,score_total,score_max_total,latitude,longitude
0,0,William hospital,Southern,Pujehun,New York,Jasonfort,community_health_post,government_public,peri_urban,yes,...,1.0,1.5,4.0,1.0,10.5,19.5,48.5,102.0,60.234598,-90.157892
1,1,Kathryn Hospital/CHC,Southern,Bo,Arizona,West Jamesshire,community_health_post,government_public,rural,yes,...,0.0,1.0,1.5,0.0,5.5,21.0,35.5,102.0,21.2454855,-107.509120
2,2,Jessica Hospital,Southern,Bo,New Mexico,Robertsside,community_health_post,government_public,rural,yes,...,1.0,1.5,2.0,0.0,10.0,33.5,55.5,102.0,-0.178657,-9.190029
3,3,Lauren CHC,Southern,Bonthe,Missouri,Port Christopher,mch_post,government_public,peri_urban,no,...,0.5,0.0,4.0,2.0,12.0,34.5,57.5,102.0,35.856989,-87.512790
4,4,Tammy CHC,Southern,Bonthe,Virginia,Reneeton,community_health_post,government_public,peri_urban,no,...,0.5,1.5,4.0,3.0,12.0,29.5,55.0,102.0,54.7148835,150.744813


In [666]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 0 to 144
Columns: 172 entries, number to fac_name
dtypes: category(109), float64(56), int64(1), object(6)
memory usage: 107.7+ KB


In [453]:
len(data['facility_name'].unique())

124

In [452]:
data['region_province'].value_counts()

Southern           37
Eastern            25
North-Western      25
Northern           25
Western            12
Nothern            10
Western Area        7
Northern Region     1
Place               1
Name: region_province, dtype: int64

In [454]:
m_data['Region_Province'].value_counts()

southern      421
eastern       311
northern      302
north-west    213
western       157
Name: Region_Province, dtype: int64

In [688]:
#rectify the regions_province for uniformity in the health centre dataset
data['region_province']=data['region_province'].replace({'Nothern':'Northern', 'Western Area':'Western', 'Nothern Region':'Northern','Northern Region':'Northern', 'Place':'North-West','North-Western':'North-West'})

In [39]:
m_data[m_data['Facility_name'] =='Dustin CHP']

,Number,Region_Province,District,Chiefdom,Facility_name,Type,Source
474,475,North-West,Kambia District,Connecticut,Dustin CHP,MCHP,MoHS
643,644,Eastern,Kenema,Wyoming,Dustin CHP,MCHP,MoHS


In [690]:
data['district'].value_counts().sort_index()

Bo                    12
Bombali               10
Bonthe                 9
District               1
Falaba                 8
Kailahun               8
Kambia                 8
Karene                 8
Kenema                 9
Koinadugu             10
Kono                   8
Moyamba                8
Port Loko              9
Pujehun                8
Tonkolili              8
Western Area Rural     8
Western Area Urban     7
western Area Urban     4
Name: district, dtype: int64

In [691]:
#relabel district names to conform with the master data
data['district']=data['district'].replace({'western Area Urban':'Western Area Urban','District':'Kambia'})

In [692]:
#relabel district names to conform with the master data
data['district']=data['district'].replace({'Bo':'Bo district','Bombali':'Bombali district', 'Kambia':'Kambia district','Kono':'Kono district','Pujehun':'Pujehun district','Western Area Urban':'Western Area Urban district','Western Area Rural':'Western Area Rural district'})

In [693]:
m_data['District'].value_counts().sort_index()

bo district                    147
bombali district                90
bonthe                          82
falaba                          41
kailahun                        89
kambia district                 71
karene                          57
kenema                         125
koinadugu                       50
kono district                   97
moyamba                        106
port loko                      102
pujehun district                85
tonkolili                      105
western area rural district     67
western area urban district     90
Name: District, dtype: int64

In [45]:
len(data['chiefdom'].value_counts())

42

In [46]:
len(m_data['Chiefdom'].value_counts())

51

In [634]:
data['chiefdom'].value_counts().sort_index()

Alabama            1
Alaska             6
Arizona            1
Arkansas           1
California         5
Colorado           5
Connecticut        2
Delaware           3
Florida            5
Georgia            8
Idaho              2
Illinois           1
Indiana            2
Iowa               3
Kansas             2
Kentucky           8
Louisiana          4
Maryland           4
Massachusetts      6
Michigan           4
Minnesota          1
Mississippi        6
Missouri           7
Montana            1
Nevada             2
New Jersey         3
New Mexico         1
New York           4
North Carolina     1
Ohio               1
Oklahoma           2
Oregon             5
Pennsylvania       1
Rhode Island       7
South Carolina     1
South Dakota       1
Tennessee         10
Texas              2
Virginia           6
Washington         3
West Virginia      3
Wisconsin          2
Name: chiefdom, dtype: int64

In [635]:
m_data['Chiefdom'].value_counts().sort_index()

Alabama           12
Alaska            22
Arizona           10
Arkansas          57
Badjia             1
California        11
Colorado          21
Connecticut       37
Delaware          37
Florida           50
Georgia           15
Hawaii            27
Idaho             24
Illinois          61
Indiana           13
Iowa              57
Kansas            25
Kentucky          57
Louisiana         32
Maine             23
Maryland          16
Massachusetts     33
Michigan          22
Minnesota          9
Mississippi       47
Missouri          11
Montana           20
Nebraska          47
Nevada            25
New Hampshire     25
New Jersey        46
New Mexico        24
New York          27
North Carolina     9
North Dakota      17
Ohio               9
Oklahoma          38
Oregon             9
Pennsylvania      29
Rhode Island      55
South Carolina    44
South Dakota      18
Tennessee         25
Texas             25
Utah              18
Vermont           10
Virginia          26
Washington   

The column 'chiefdom' seems to be uniformly labelled in both datasets

Data Matching
string similarity algorithmw will be prioritised for this

In [694]:
#uniformly make to lower shared columns in both datasets
m_data[['Region_Province','District','Chiefdom','Facility_name']]=m_data[['Region_Province','District','Chiefdom','Facility_name']].astype(str).apply(lambda x: x.str.lower())

In [695]:
#uniformly make to lower shared columns in both datasets
data[['facility_name','region_province','district','chiefdom']]=data[['facility_name','region_province','district','chiefdom']].astype(str).apply(lambda x: x.str.lower())

In [696]:
master_facility = m_data['Facility_name'].value_counts().to_dict()
data_facility = data['facility_name'].value_counts().to_dict()

In [697]:
fa =0
for i in master_facility.keys():
    for j in data_facility.keys():
        if i ==j:
            fa+=1

In [698]:
#this is the union set of hospital names that are in both the master facility and health facilty
fa

56

In [699]:
health_set =set(())
master_set=set(())
for i in data_facility.keys():
    health_set.add(i)
for i in master_facility.keys():
    master_set.add(i)

list(health_set.difference(master_set))


['cheryl hospital/chc',
 'sean hospital',
 'george chc',
 'dennis mchp',
 'jorge mchp',
 'lauren chc',
 'joseph clinic',
 'justin clinic',
 'nathan hospital/chc',
 'kelsey regional hospital',
 'levi chc',
 'steven tertiary hospital',
 'lance regional hospital',
 'trevor tertiary hospital',
 'monica clinic',
 'nicole tertiary hospital',
 'sherry chc',
 'dalton hospital',
 'glenn chp',
 'lindsey mchp',
 'melinda hospital/chc',
 'dawn clinic',
 'adam mchp',
 'walter mchp',
 'gene chp',
 'tammy hospital',
 'savannah tertiary hospital',
 'amber chp',
 'amber tertiary hospital',
 'grant chp',
 'jamie mchp',
 'carlos chp',
 'bradley mchp',
 'edward tertiary hospital',
 'selena hospital/chc',
 'jeremiah clinic',
 'melissa clinic',
 'lee chp',
 'joanna clinic',
 'charles clinic',
 'andrew chp',
 'kathryn hospital/chc',
 'nathaniel regional hospital',
 'carolyn mchp',
 'rebekah tertiary hospital',
 'zoe hospital/chc',
 'kayla tertiary hospital',
 'charles tertiary hospital',
 'tanya clinic',
 'j

In [778]:
#create new column of the first value of the facility name or in some instances the first 2 values for ease ofcomparison
data['fac_name']=data['facility_name'].replace({r'/':' ','chc':'','mchp':'','chp':'','clinic':'','hospital':''},regex=True).str.strip()
m_data['facName']=m_data['Facility_name'].replace({r'/':' ','chc':'','mchp':'','chp':'','clinic':'','hospital':''},regex=True).str.strip()

In [779]:
#create two datasets with one containing a list whose faility name are contained in the master list and the oppposite
data_rd = data[data['facility_name'].isin(list(health_set.difference(master_set)))]
data_ok = data[-data['facility_name'].isin(list(health_set.difference(master_set)))]


In [780]:
# make numbers column an index for use in jarowinkler algorithm
data_rd = data_rd.set_index('number')


In [765]:
# make numbers column an index for use in jarowinkler algorithm
m_data = m_data.set_index('Number')

In [704]:
m_data.head()

,Region_Province,District,Chiefdom,Facility_name,Type,Source,facName
Number,,,,,,,
1,southern,bo district,badjia,ngelehun chc,CHC,MoHS,ngelehun
2,southern,bo district,florida,victoria chp,MCHP,MoHS,victoria
3,southern,bo district,louisiana,kristina chc,CHC,MoHS,kristina
4,southern,bo district,louisiana,james regional hospital,MCHP,MoHS,james regional
5,southern,bo district,louisiana,jake regional hospital,MCHP,MoHS,jake regional


In [705]:
data.shape

(143, 172)

In [768]:
data_rd.head()

,number,facility_name,region_province,district,chiefdom,facility_location,facility_type,managing_authority,setting,outpatient_only,...,score_infection_prevention_and_control_waste_collection_and_disposal,score_infection_prevention_and_control_water_sanitation_and_hygiene,score_infection_prevention_and_control_disinfection_and_sterilization,score_infection_prevention_and_control,score_logistics_patient_and_sample_transfer,score_total,score_max_total,latitude,longitude,fac_name
1,1,kathryn hospital/chc,southern,bo district,arizona,West Jamesshire,community_health_post,government_public,rural,yes,...,1.0,1.5,0.0,5.5,21.0,35.5,102.0,21.2454855,-107.509120,kathryn
3,3,lauren chc,southern,bonthe,missouri,Port Christopher,mch_post,government_public,peri_urban,no,...,0.0,4.0,2.0,12.0,34.5,57.5,102.0,35.856989,-87.512790,lauren
6,6,bryan clinic,southern,pujehun district,michigan,Adammouth,community_health_post,government_public,peri_urban,yes,...,0.0,1.0,2.0,8.5,21.0,53.0,102.0,61.8040035,28.191714,bryan
7,7,cheryl hospital/chc,southern,bonthe,nevada,Bryanchester,community_health_centre,government_public,peri_urban,no,...,1.0,3.0,1.0,7.0,15.0,38.0,102.0,86.9708915,173.527045,cheryl
8,8,brandi hospital,southern,moyamba,florida,Port Kristinville,community_health_centre,government_public,peri_urban,yes,...,4.0,2.5,4.0,19.5,25.0,65.5,102.0,77.8973365,70.112618,brandi


In [964]:
fig = px.scatter_geo(data_frame=data, lat=data['latitude'],lon=data['longitude'])
fig.show()

In [781]:
indexer = rd.Index()
indexer.block('chiefdom','Chiefdom')
indexer.block(left_on ='district' , right_on = 'District')
indexer.block('region_province', 'Region_Province')


<Index>

In [782]:
candidates = indexer.index(data_rd,m_data)
len(candidates)

24139

In [572]:
data_rd.head()

,facility_name,region_province,district,chiefdom,facility_location,facility_type,managing_authority,setting,outpatient_only,number_consultation_rooms,...,score_infection_prevention_and_control_water_sanitation_and_hygiene,score_infection_prevention_and_control_disinfection_and_sterilization,score_infection_prevention_and_control,score_logistics_patient_and_sample_transfer,score_total,score_max_total,latitude,longitude,chefdom,fac_name
number,,,,,,,,,,,,,,,,,,,,,
1,kathryn hospital/chc,southern,bo district,arizona,West Jamesshire,community_health_post,government_public,rural,yes,1.0,...,1.5,0.0,5.5,21.0,35.5,102.0,21.2454855,-107.509120,arizona,kathryn
3,lauren chc,southern,bonthe,missouri,Port Christopher,mch_post,government_public,peri_urban,no,1.0,...,4.0,2.0,12.0,34.5,57.5,102.0,35.856989,-87.512790,missouri,lauren
6,bryan clinic,southern,pujehun district,michigan,Adammouth,community_health_post,government_public,peri_urban,yes,1.0,...,1.0,2.0,8.5,21.0,53.0,102.0,61.8040035,28.191714,michigan,bryan
7,cheryl hospital/chc,southern,bonthe,nevada,Bryanchester,community_health_centre,government_public,peri_urban,no,1.0,...,3.0,1.0,7.0,15.0,38.0,102.0,86.9708915,173.527045,nevada,cheryl
8,brandi hospital,southern,moyamba,florida,Port Kristinville,community_health_centre,government_public,peri_urban,yes,2.0,...,2.5,4.0,19.5,25.0,65.5,102.0,77.8973365,70.112618,florida,brandi


In [783]:
#get the abbreviations for the type of hospital
data_rd['ab_fac_type']=data_rd['facility_type'].replace({'mch_post':'MCHP','community_health_post':'CHP','community_health_centre':'CHC'})

In [784]:
compare = rd.Compare()
# compare.exact('region_province','Region_Province',label = 'Reg')
# compare.exact('district','District', label='dist')
compare.string('fac_name','facName',label = 'facName',method='jarowinkler',threshold = 0.66)
# compare.exact('chiefdom','Chiefdom',label='chief')
compare.string('facility_name','Facility_name',label='facilityN',method='jarowinkler',threshold = 0.71)
# compare.exact('ab_fac_type','Type',label = 'fac_type')




<Compare>

In [785]:
features = compare.compute(candidates,data_rd,m_data)

In [786]:
features.sum(axis=1).value_counts().sort_index()

0.0    20368
1.0     3028
2.0      743
dtype: int64

In [756]:
features

facName  facilityN
number Number                    
1      1           0.0        0.0
       2           0.0        0.0
       3           0.0        1.0
       4           0.0        0.0
       5           0.0        0.0
...                ...        ...
142    1243        0.0        0.0
       1244        0.0        0.0
       1245        0.0        0.0
       1246        0.0        0.0
       1247        0.0        0.0

[24139 rows x 2 columns]

In [787]:
potential_matches=features[features.sum(axis=1)>1].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'facName':'facilityN'].sum(axis=1)

In [788]:
potential_matches

,number,Number,facName,facilityN,Score
0,1,51,1.0,1.0,2.0
1,1,116,1.0,1.0,2.0
2,1,126,1.0,1.0,2.0
3,1,240,1.0,1.0,2.0
4,1,291,1.0,1.0,2.0
...,...,...,...,...,...
738,140,1122,1.0,1.0,2.0
739,142,229,1.0,1.0,2.0
740,142,568,1.0,1.0,2.0
741,142,729,1.0,1.0,2.0


In [789]:
len(potential_matches['number'].unique())

73

In [595]:
len(data_rd.index)

73

In [790]:
#comparison to generate maximum value list
m_compare = rd.Compare()
m_compare.string('fac_name','facName',label = 'facName',method='jarowinkler')
m_compare.string('facility_name','Facility_name',label='facilityN',method='jarowinkler')
m_features = m_compare.compute(candidates,data_rd,m_data)
m_potential_matches=m_features[m_features.sum(axis=1)>0].reset_index()
m_potential_matches['Score'] = m_potential_matches.loc[:, 'facName':'facilityN'].sum(axis=1)
pot_match=m_potential_matches[m_potential_matches['Score']>=1.37]
pot_match=m_potential_matches[m_potential_matches['Score']>=1.37]
max_score = m_potential_matches.groupby('number')['Score'].max().reset_index()
#get the max score  value to get the most probable match
match = pot_match[pot_match['Score'].isin(max_score['Score'].tolist())].groupby('number').max()


In [791]:
match.head(20)

,Number,facName,facilityN,Score
number,,,,
1,946,0.942857,0.940000,1.882857
3,254,1.000000,0.866667,1.866667
6,932,1.000000,0.972222,1.905556
7,132,0.880000,0.842895,1.722895
8,267,0.857778,0.963175,1.820952
11,552,0.880000,0.847810,1.727810
12,742,0.688889,0.952381,1.641270
13,599,0.961905,0.778462,1.740366
14,394,0.792599,0.818075,1.610674


In [762]:
#sample comparison
data_rd.loc[20,:]

facility_name           steven tertiary hospital
region_province                          western
district             western area urban district
chiefdom                                illinois
facility_location                   Colleenburgh
                                ...             
score_max_total                            102.0
latitude                              -9.2838505
longitude                              51.933410
fac_name                         steven tertiary
ab_fac_type                                  CHC
Name: 20, Length: 172, dtype: object

In [763]:
#master data comparative
m_data.loc[1347,:]

Region_Province                       western 
District           western area urban district
Chiefdom                              illinois
Facility_name         george tertiary hospital
Type                                       CHP
Source                                    MoHS
facName                        george tertiary
Name: 1347, dtype: object

In [325]:
m_data.head()

,Region_Province,District,Chiefdom,Facility_name,Type,Source,facName
Number,,,,,,,
1,southern,bo district,badjia,ngelehun chc,CHC,MoHS,ngelehun
2,southern,bo district,florida,victoria chp,MCHP,MoHS,victoria
3,southern,bo district,louisiana,kristina chc,CHC,MoHS,kristina
4,southern,bo district,louisiana,james regional hospital,MCHP,MoHS,james regional
5,southern,bo district,louisiana,jake regional hospital,MCHP,MoHS,jake regional


In [792]:
data_rd['hospital_lookup'] = data_rd[['facility_name','fac_name']].apply(lambda x: '_'.join(x), axis=1)
m_data['master_hosp_lookup'] = m_data[['Facility_name','facName']].apply(lambda x: '_'.join(x), axis=1)
hosp_lookup = data_rd[['hospital_lookup']].reset_index()
m_hosp_lookup = m_data[['master_hosp_lookup']].reset_index()

In [793]:
account_merge = potential_matches.merge(hosp_lookup, how='left')
final_merge = account_merge.merge(m_hosp_lookup, how='left')

In [794]:
final_merge.head(30)

,number,Number,facName,facilityN,Score,hospital_lookup,master_hosp_lookup
0,1,51,1.0,1.0,2.0,kathryn hospital/chc_kathryn,mary hospital/chc_mary
1,1,116,1.0,1.0,2.0,kathryn hospital/chc_kathryn,karina mchp_karina
2,1,126,1.0,1.0,2.0,kathryn hospital/chc_kathryn,karen hospital/chc_karen
3,1,240,1.0,1.0,2.0,kathryn hospital/chc_kathryn,heather hospital_heather
4,1,291,1.0,1.0,2.0,kathryn hospital/chc_kathryn,kara hospital_kara
5,1,313,1.0,1.0,2.0,kathryn hospital/chc_kathryn,kathryn mchp_kathryn
6,1,864,1.0,1.0,2.0,kathryn hospital/chc_kathryn,martin hospital/chc_martin
7,1,898,1.0,1.0,2.0,kathryn hospital/chc_kathryn,keith hospital_keith
8,1,904,1.0,1.0,2.0,kathryn hospital/chc_kathryn,karina hospital_karina
9,1,930,1.0,1.0,2.0,kathryn hospital/chc_kathryn,anthony hospital/chc_anthony


using fuzzywuzzy

In [488]:
for  i in data_facility.keys():
    process.extract(i,list(m_data['Facility_name'].unique()))

In [506]:
process.extract('JOANNA CLINIC',m_data['Facility_name'].unique())

[('ANN CLINIC', 87),
 ('JOANNA TERTIARY HOSPITAL', 86),
 ('HANNAH CLINIC', 85),
 ('JOHN CLINIC', 83),
 ('WANDA CLINIC', 80)]

In [644]:
for i in process.extractBests('EDWARD',m_data['Facility_name'].unique(),limit = 10):
    print(i)
    # if data[z]['some_column']==master_data['some_column'] and data[z]['some_other_column']

('EDWARD MCHP', 90)
('EDWARD CHP', 90)
('BERNARD HOSPITAL', 60)
('GLENDA REGIONAL HOSPITAL', 60)
('LEONARD TERTIARY HOSPITAL', 60)
('EDWIN CHP', 47)
('EDWIN CHC', 47)
('MATTHEW REGIONAL HOSPITAL', 45)
('HEATHER CHP', 45)
('KENDRA TERTIARY HOSPITAL', 45)


In [976]:
data_rd = data_rd.reset_index()
m_data = m_data.reset_index()

In [958]:
ff_features=fuzzymatcher.fuzzy_left_join(data_rd,m_data,left_on=['fac_name','facility_name'],
                                right_on=['facName','Facility_name'],left_id_col = 'number',
            right_id_col = 'Number')

In [963]:
ff_features.sort_values(by='best_match_score',ascending=False).tail(30)

,best_match_score,__id_left,__id_right,number,facility_name,region_province,district,chiefdom,facility_location,facility_type,...,fac_name,ab_fac_type,Number,Region_Province,District,Chiefdom,Facility_name,Type,Source,facName
4420,0.284663,137,1320,137,LEE CHC,NORTH-WEST,PORT LOKO,FLORIDA,Valerieborough,mch_post,...,LEE,MCHP,1320,WESTERN,WESTERN AREA URBAN DISTRICT,NEVADA,LEE REGIONAL HOSPITAL,Clinic,MoHS,LEE REGIONAL
3625,0.282720,104,153,104,AMANDA TERTIARY HOSPITAL,WESTERN,WESTERN AREA RURAL DISTRICT,LOUISIANA,Lake Alicemouth,community_health_post,...,AMANDA TERTIARY,CHP,153,NORTHERN,BOMBALI DISTRICT,FLORIDA,AMANDA HOSPITAL,MCHP,MoHS,AMANDA
4370,0.276940,135,286,135,LINDSEY MCHP,EASTERN,KONO DISTRICT,CALIFORNIA,Rodriguezbury,mch_post,...,LINDSEY,MCHP,286,SOUTHERN,BONTHE,MAINE,LINDSAY MCHP,MCHP,MoHS,LINDSAY
1889,0.274682,57,85,57,LISA TERTIARY HOSPITAL,NORTHERN,BOMBALI DISTRICT,DELAWARE,Gibsonstad,community_health_post,...,LISA TERTIARY,CHP,85,SOUTHERN,BO DISTRICT,RHODE ISLAND,LISA HOSPITAL,CHP,MoHS,LISA
2756,0.274682,80,85,80,LISA TERTIARY HOSPITAL,EASTERN,KAILAHUN,VIRGINIA,East Debra,mch_post,...,LISA TERTIARY,MCHP,85,SOUTHERN,BO DISTRICT,RHODE ISLAND,LISA HOSPITAL,CHP,MoHS,LISA
0,0.274341,1,390,1,KATHRYN HOSPITAL/CHC,SOUTHERN,BO DISTRICT,ARIZONA,West Jamesshire,community_health_post,...,KATHRYN,CHP,390,EASTERN,KAILAHUN,OKLAHOMA,KATRINA HOSPITAL/CHC,CHP,MoHS,KATRINA
1631,0.272775,51,1133,51,EDWARD TERTIARY HOSPITAL,WESTERN,WESTERN AREA RURAL DISTRICT,MISSISSIPPI,Mitchellport,community_health_centre,...,EDWARD TERTIARY,CHC,1133,SOUTHERN,PUJEHUN DISTRICT,HAWAII,EDWARD CHP,MCHP,MoHS,EDWARD
4174,0.265887,126,137,126,BILLY CLINIC,SOUTHERN,BONTHE,MISSOURI,West Davidview,community_health_centre,...,BILLY,CHC,137,SOUTHERN,BO DISTRICT,IOWA,BILLY CHP,CHC,MoHS,BILLY
3094,0.264290,90,757,90,KAYLA TERTIARY HOSPITAL,SOUTHERN,BONTHE,CALIFORNIA,New Lisaville,mch_post,...,KAYLA TERTIARY,MCHP,757,EASTERN,KONO DISTRICT,MISSOURI,KAYLA HOSPITAL,MCHP,MoHS,KAYLA
2312,0.263237,69,28,69,CHARLES TERTIARY HOSPITAL,WESTERN,WESTERN AREA RURAL DISTRICT,RHODE ISLAND,Paulfort,mch_post,...,CHARLES TERTIARY,MCHP,28,SOUTHERN,BO DISTRICT,LOUISIANA,CHARLES HOSPITAL,CHP,MoHS,CHARLES


In [749]:
m_data.loc[312,:]

Number                      313
Region_Province        SOUTHERN
District                 BONTHE
Chiefdom                  MAINE
Facility_name      KATHRYN MCHP
Type                       MCHP
Source                     MoHS
facName                KATHRYN 
Name: 312, dtype: object

In [765]:
for i in data_rd['facility_name'].replace({r'/':' ','CHC':'','MCHP':'','CHP':'','CLINIC':'','HOSPITAL':''},regex=True).str.strip().to_list():
    print(process.extractOne(i,m_data['Facility_name'].replace({r'/':' ','CHC':'','MCHP':'','CHP':'','CLINIC':'','HOSPITAL':''},regex=True).str.strip().to_list()))

('KATHRYN', 100)
('LAUREN', 100)
('BRYAN', 100)
('CHERYL', 100)
('BRANDI REGIONAL', 90)
('KELSEY', 90)
('TAMMY', 100)
('KARINA', 100)
('ANNA', 90)
('CHARLES', 100)
('RANDY', 100)
('STEVEN', 90)
('DAWN', 100)
('ROBERTO', 60)
('JOANNA', 100)
('TREVOR', 100)
('REBECCA TERTIARY', 88)
('CARLOS', 100)
('BRITTANY', 80)
('AMBER', 90)
('MIA', 90)
('AMBER', 100)
('DALTON', 77)
('AMBER', 100)
('JOSEPH', 100)
('GLEN', 89)
('JOANNA', 100)
('ADAM', 100)
('EDWARD', 90)
('DALTON', 100)
('ANDREW', 100)
('GEORGE', 100)
('LISA', 90)
('JUSTIN', 100)
('NATHAN', 100)
('WALTER', 100)
('NATHAN REGIONAL', 91)
('LANCE', 90)
('CHARLES', 90)
('BRITTANY', 80)
('JAMIE', 100)
('EUGENE', 90)
('EILEEN', 90)
('CAROLYN', 100)
('NICOLE', 90)
('LISA', 90)
('TREVOR', 90)
('VALERIE', 68)
('MELISSA', 100)
('GEORGE', 73)
('KAYLA', 90)
('JOSHUA', 100)
('SELENA', 100)
('DEAN', 100)
('SEAN', 100)
('AMANDA', 100)
('MONICA', 100)
('AMANDA', 90)
('RYAN', 67)
('GEORGE', 100)
('MELISSA', 100)
('SHAWN', 100)
('BRADLEY', 100)
('MARILYN

In [ ]:
#convert relevant columns to categorical variables
data[data.columns[48:150]]=data[data.columns[48:150]].astype('category')

In [795]:
data.head()

,number,facility_name,region_province,district,chiefdom,facility_location,facility_type,managing_authority,setting,outpatient_only,...,score_infection_prevention_and_control_waste_collection_and_disposal,score_infection_prevention_and_control_water_sanitation_and_hygiene,score_infection_prevention_and_control_disinfection_and_sterilization,score_infection_prevention_and_control,score_logistics_patient_and_sample_transfer,score_total,score_max_total,latitude,longitude,fac_name
0,0,william hospital,southern,pujehun district,new york,Jasonfort,community_health_post,government_public,peri_urban,yes,...,1.5,4.0,1.0,10.5,19.5,48.5,102.0,60.234598,-90.157892,william
1,1,kathryn hospital/chc,southern,bo district,arizona,West Jamesshire,community_health_post,government_public,rural,yes,...,1.0,1.5,0.0,5.5,21.0,35.5,102.0,21.2454855,-107.509120,kathryn
2,2,jessica hospital,southern,bo district,new mexico,Robertsside,community_health_post,government_public,rural,yes,...,1.5,2.0,0.0,10.0,33.5,55.5,102.0,-0.178657,-9.190029,jessica
3,3,lauren chc,southern,bonthe,missouri,Port Christopher,mch_post,government_public,peri_urban,no,...,0.0,4.0,2.0,12.0,34.5,57.5,102.0,35.856989,-87.512790,lauren
4,4,tammy chc,southern,bonthe,virginia,Reneeton,community_health_post,government_public,peri_urban,no,...,1.5,4.0,3.0,12.0,29.5,55.0,102.0,54.7148835,150.744813,tammy


In [802]:
data.to_csv('/home/dr/Downloads/health_cleaned.csv')

In [803]:
data.dtypes.to_frame().to_csv('/home/dr/Downloads/dictionary.csv')